In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
# Python libs
import numpy as np
import pandas as pd
from os.path import isfile, join
import os.path
import re
from google.colab import drive
import sys
import xgboost as xgb

In [0]:
steps_10_mins = 1
steps_20_mins = 2
steps_30_mins = 3
steps_1_hour = 6
steps_2_hours = 12
steps_12_hours = 12 * steps_1_hour
steps_24_hours = 24 * steps_1_hour

column_year = 'year'
column_month = 'month'
column_hour = 'hour'

MASTER_DATASET = 'model_data_all_features_classified.csv.zip'

In [0]:
pd.options.display.float_format = '{:,.2f}'.format # Limit output to 3 decimal places.
pd.set_option('display.max_columns', None)

In [0]:
data_path = '../gdrive/My Drive/bise_data'

In [5]:
drive.mount('/gdrive')
#drive.mount('/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [6]:
file_master = os.path.join(data_path, MASTER_DATASET)
df_master = pd.read_csv(file_master, index_col=0)
df_master.head()

,datetime,original_index,year,month,bise_event,rule3_episode,rule4_episode,rule3_duration,rule4_duration,rule3_label,rule4_label,manual_bise_x,manual_bise_y,manual_duration_class,rule4_episode_1_0,rule4_duration_class,KLO_Air_temp,KLO_Hum,KLO_Hum_delta_3,KLO_Hum_delta_6,KLO_QNH,KLO_Wind_dir,KLO_Wind_dir_delta,KLO_Wind_dir_delta-3,KLO_Wind_dir_delta-6,KLO_Wind_dir_spread30,KLO_Wind_dir_spread60,KLO_Wind_max_EW,KLO_Wind_max_NS,KLO_Wind_max,KLO_Wind_max_delta_3,KLO_Wind_max_delta_6,KLO_Wind_speed_EW,KLO_Wind_speed_NS,KLO_Wind_speed,KLO_Wind_speed_delta_3,KLO_Wind_speed_delta_6,BIZ_Air_temp,BIZ_Hum,BIZ_Hum_delta_3,BIZ_Hum_delta_6,BIZ_QNH,BIZ_Wind_dir,BIZ_Wind_dir_delta,BIZ_Wind_dir_delta-3,BIZ_Wind_dir_delta-6,BIZ_Wind_dir_spread30,BIZ_Wind_dir_spread60,BIZ_Wind_max_EW,BIZ_Wind_max_NS,BIZ_Wind_max,BIZ_Wind_max_delta_3,BIZ_Wind_max_delta_6,BIZ_Wind_speed_EW,BIZ_Wind_speed_NS,BIZ_Wind_speed,BIZ_Wind_speed_delta_3,BIZ_Wind_speed_delta_6,BUS_Air_temp,BUS_Hum,BUS_Hum_delta_3,BUS_Hum_delta_6,BUS_QNH,BUS_Wind_dir,BUS_Wind_dir_delta,BUS_Wind_dir_delta-3,BUS_Wind_dir_delta-6,BUS_Wind_dir_spread30,BUS_Wind_dir_spread60,BUS_Wind_max_EW,BUS_Wind_max_NS,BUS_Wind_max,BUS_Wind_max_delta_3,BUS_Wind_max_delta_6,BUS_Wind_speed_EW,BUS_Wind_speed_NS,BUS_Wind_speed,BUS_Wind_speed_delta_3,BUS_Wind_speed_delta_6,GUT_Air_temp,GUT_Hum,GUT_Hum_delta_3,GUT_Hum_delta_6,GUT_QNH,GUT_Wind_dir,GUT_Wind_dir_delta,GUT_Wind_dir_delta-3,GUT_Wind_dir_delta-6,GUT_Wind_dir_spread30,GUT_Wind_dir_spread60,GUT_Wind_max_EW,GUT_Wind_max_NS,GUT_Wind_max,GUT_Wind_max_delta_3,GUT_Wind_max_delta_6,GUT_Wind_speed_EW,GUT_Wind_speed_NS,GUT_Wind_speed,GUT_Wind_speed_delta_3,GUT_Wind_speed_delta_6,KOP_Air_temp,KOP_Hum,KOP_Hum_delta_3,KOP_Hum_delta_6,KOP_QNH,KOP_Wind_dir,KOP_Wind_dir_delta,KOP_Wind_dir_delta-3,KOP_Wind_dir_delta-6,KOP_Wind_dir_spread30,KOP_Wind_dir_spread60,KOP_Wind_max_EW,KOP_Wind_max_NS,KOP_Wind_max,KOP_Wind_max_delta_3,KOP_Wind_max_delta_6,KOP_Wind_speed_EW,KOP_Wind_speed_NS,KOP_Wind_speed,KOP_Wind_speed_delta_3,KOP_Wind_speed_delta_6,MOA_Air_temp,MOA_Hum,MOA_Hum_delta_3,MOA_Hum_delta_6,MOA_QNH,MOA_Wind_dir,MOA_Wind_dir_delta,MOA_Wind_dir_delta-3,MOA_Wind_dir_delta-6,MOA_Wind_dir_spread30,MOA_Wind_dir_spread60,MOA_Wind_max_EW,MOA_Wind_max_NS,MOA_Wind_max,MOA_Wind_max_delta_3,MOA_Wind_max_delta_6,MOA_Wind_speed_EW,MOA_Wind_speed_NS,MOA_Wind_speed,MOA_Wind_speed_delta_3,MOA_Wind_speed_delta_6,MUB_Air_temp,MUB_Hum,MUB_Hum_delta_3,MUB_Hum_delta_6,MUB_QNH,MUB_Wind_dir,MUB_Wind_dir_delta,MUB_Wind_dir_delta-3,MUB_Wind_dir_delta-6,MUB_Wind_dir_spread30,MUB_Wind_dir_spread60,MUB_Wind_max_EW,MUB_Wind_max_NS,MUB_Wind_max,MUB_Wind_max_delta_3,MUB_Wind_max_delta_6,MUB_Wind_speed_EW,MUB_Wind_speed_NS,MUB_Wind_speed,MUB_Wind_speed_delta_3,MUB_Wind_speed_delta_6,PAY_Air_temp,PAY_Hum,PAY_Hum_delta_3,PAY_Hum_delta_6,PAY_QNH,PAY_Wind_dir,PAY_Wind_dir_delta,PAY_Wind_dir_delta-3,PAY_Wind_dir_delta-6,PAY_Wind_dir_spread30,PAY_Wind_dir_spread60,PAY_Wind_max_EW,PAY_Wind_max_NS,PAY_Wind_max,PAY_Wind_max_delta_3,PAY_Wind_max_delta_6,PAY_Wind_speed_EW,PAY_Wind_speed_NS,PAY_Wind_speed,PAY_Wind_speed_delta_3,PAY_Wind_speed_delta_6,PUY_Air_temp,PUY_Hum,PUY_Hum_delta_3,PUY_Hum_delta_6,PUY_QNH,PUY_Wind_dir,PUY_Wind_dir_delta,PUY_Wind_dir_delta-3,PUY_Wind_dir_delta-6,PUY_Wind_dir_spread30,PUY_Wind_dir_spread60,PUY_Wind_max_EW,PUY_Wind_max_NS,PUY_Wind_max,PUY_Wind_max_delta_3,PUY_Wind_max_delta_6,PUY_Wind_speed_EW,PUY_Wind_speed_NS,PUY_Wind_speed,PUY_Wind_speed_delta_3,PUY_Wind_speed_delta_6,REH_Air_temp,REH_Hum,REH_Hum_delta_3,REH_Hum_delta_6,REH_QNH,REH_Wind_dir,REH_Wind_dir_delta,REH_Wind_dir_delta-3,REH_Wind_dir_delta-6,REH_Wind_dir_spread30,REH_Wind_dir_spread60,REH_Wind_max_EW,REH_Wind_max_NS,REH_Wind_max,REH_Wind_max_delta_3,REH_Wind_max_delta_6,REH_Wind_speed_EW,REH_Wind_speed_NS,REH_Wind_speed,REH_Wind_speed_delta_3,REH_Wind_speed_delta_6,SHA_Air_temp,SHA_Hum,SHA_Hum_delta_3,SHA_Hum_delta_6,SHA_QNH,SHA_Wind_dir,SHA_Wind_dir_delta,SHA_Wind_dir_delta-3,SHA_Wind_dir_delta-6,SHA_Wind_dir_spre

In [0]:
def xgb_cross_validation(x_train, y_train, **hyperparams):
  n_folds = 5
  early_stopping = 10
  
  xg_train = xgb.DMatrix(x_train, label=y_train);

  return xgb.cv(hyperparams, xg_train, 5000, nfold=n_folds, early_stopping_rounds=early_stopping, verbose_eval=1, seed=42, metrics={'mlogloss'}) 
    

In [14]:
df_base = df_master[6:]
df_base_train = df_base[df_base.year < 2018]
df_base_test = df_base[df_base.year >= 2018]

print(df_base_train.shape, df_base_test.shape)

(210378, 832) (52417, 832)


In [0]:
target_col = 'rule4_duration_class'

In [0]:
feat_klo = [c for c in df_base.columns if re.search('KLO', c)]
df_train_x_klo = df_base_train[feat_klo]
df_train_y_klo = df_base_train[target_col]
df_test_x_klo = df_base_train[feat_klo]
df_test_y_klo = df_base_train[target_col]

In [0]:
hyperparams = {'subsample': 0.4, 'scale_pos_weight': 1, 'n_estimators': 50, 
               'max_depth': 5, 'learning_rate': 0.1, 'gamma': 1, 'colsample_bytree': 1,
              'objective': "multi:softprob", 'num_class': 4, 'gpu_id': 0, 'tree_method': "gpu_hist", 'eval_metric': 'mlogloss'}

  

cvr = xgb_cross_validation(df_train_x_klo, df_train_y_klo, hyperparams=hyperparams)
cvr.head()

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [0]:
cvr['test-rmse-mean'].argmin()

In [13]:
gridsearch_params = [
    (max_depth, min_child_weight, subsample, colsample, eta)
    for max_depth in range(2, 5,12)
    for min_child_weight in range(1, 5,8)
    for subsample in [i/10. for i in range(7,11)]
    for colsample in [i/10. for i in range(7,11)]
    for eta in [.3, .1, .05, .01, .005]
    
]

for max_depth, min_child_weight, subsample, colsample, eta in gridsearch_params:
  print("CV with max_depth={}, min_child_weight={}, subsample={}, colsample={}, eta={}".format(
                             max_depth, min_child_weight, subsample, colsample, eta))
    # Run CV
  cv_results = xgb_cross_validation(x_train, y_train, **hyperparams)
  
  mean_mae = cv_results['test-mae-mean'].min()
  boost_rounds = cv_results['test-mae-mean'].argmin()
  print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
  if mean_mae < min_mae:
    min_mae = mean_mae
    best_params = (max_depth, min_child_weight, subsample, colsample, eta)
print("Best params: {}, {}, {}, {}, {} MAE: {}".format(best_params[0], best_params[1], best_params[2], best_params[3], best_params[4], min_mae))



CV with max_depth=2, min_child_weight=1, subsample=0.7, colsample=0.7, eta=0.3
CV with max_depth=2, min_child_weight=1, subsample=0.7, colsample=0.7, eta=0.1
CV with max_depth=2, min_child_weight=1, subsample=0.7, colsample=0.7, eta=0.05
CV with max_depth=2, min_child_weight=1, subsample=0.7, colsample=0.7, eta=0.01
CV with max_depth=2, min_child_weight=1, subsample=0.7, colsample=0.7, eta=0.005
CV with max_depth=2, min_child_weight=1, subsample=0.7, colsample=0.8, eta=0.3
CV with max_depth=2, min_child_weight=1, subsample=0.7, colsample=0.8, eta=0.1
CV with max_depth=2, min_child_weight=1, subsample=0.7, colsample=0.8, eta=0.05
CV with max_depth=2, min_child_weight=1, subsample=0.7, colsample=0.8, eta=0.01
CV with max_depth=2, min_child_weight=1, subsample=0.7, colsample=0.8, eta=0.005
CV with max_depth=2, min_child_weight=1, subsample=0.7, colsample=0.9, eta=0.3
CV with max_depth=2, min_child_weight=1, subsample=0.7, colsample=0.9, eta=0.1
CV with max_depth=2, min_child_weight=1, sub